In [40]:
import geopandas as gpd
import json
from shapely.ops import nearest_points
import pandas as pd

In [41]:
admin2 = gpd.read_file('gadm36_ETH_shp/gadm36_ETH_3.shp')
admin2['Gadm Name 2'] = admin2['NAME_2']

In [42]:
pts3 = admin2.centroid.unary_union

In [43]:
from shapely.ops import nearest_points
# unary union of the gpd2 geomtries 

def near(point, pts=pts3):
     # find the nearest point and return the corresponding Place value
    nearest = admin2.centroid == nearest_points(point, pts)[1]
    return admin2[nearest]['Gadm Name 2'].get_values()[0]

In [44]:
maln = gpd.read_file('malnutrition_eth.geojson' )
pop = gpd.read_file("population_ETH_2018.geojson")
travel = gpd.read_file("travel_time.geojson")

In [45]:
flood = pd.read_csv('flood-model.csv')
flood = gpd.GeoDataFrame(
    flood, geometry=gpd.points_from_xy(flood.longitude, flood.latitude))

In [46]:
pop = pop[pop['Population']>0]

In [47]:
maln['centroid']=maln.geometry.apply(lambda x: x.centroid)
pop['centroid']=pop.geometry.apply(lambda x: x.centroid)
admin2['centroid']=admin2.geometry.apply(lambda x: x.centroid)

In [48]:
pop['Gadm Name 2'] = pop.apply(lambda row: near(row.centroid), axis=1)
maln['Gadm Name 2'] = maln.apply(lambda row: near(row.centroid), axis=1)

# for Flood, just use geometry, not centroid since these are just points not shapes
flood['Gadm Name 2'] = flood.apply(lambda row: near(row.geometry), axis=1)

In [49]:
del(pop['centroid'])
del(maln['centroid'])

In [52]:
pop.to_file("pop_ETH.geojson", driver="GeoJSON")
maln.to_file("maln_ETH.geojson", driver="GeoJSON")
flood.to_file("flood_ETH.geojson", driver="GeoJSON")